In [1]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [23]:
from numpy.linalg import matrix_power

def beta_cov(x, y):
    n = x.shape[0]
    beta_star = np.matmul(matrix_power((1/n)*np.matmul(x.T, x), -1),  (1/n)*np.matmul(x.T, y))
    H = (1/n)*np.matmul(x.T, x)
    print(beta_star.shape)
    print(x.shape)
    print(y.shape)
    print(x.T.shape)
    V = (np.diag(y - np.matmul(x, beta_star)))**2
    print(V.shape)
    return beta_star


rng = np.random.default_rng(1337)
x = ss.norm.rvs(size=(100, 3), random_state=rng)
eps = ss.norm.rvs(size=(100, 1), random_state=rng)
beta = ss.norm.rvs(size=(3, 1), random_state=rng)
y = x @ beta + eps
x.shape[0]

print(*beta_cov(x, y), sep='\n')

(3, 1)
(100, 3)
(100, 1)
(3, 100)
(1,)
[-1.0878702]
[-0.5688717]
[0.93436482]
